# Import Necessary Libraries

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import shap
import torch
from Explainer import *

# Load Dataset

In [15]:
data = pd.read_csv('Compas/Dataset/compas.csv')

# Explore Dataset

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

# Data Preprocessing

In [16]:
data = data.dropna(subset=['c_jail_in', 'c_jail_out'])

In [17]:
# filter similar to propublica
data = data[
    (data["days_b_screening_arrest"] <= 30)
    & (data["days_b_screening_arrest"] >= -30)
    & (data["is_recid"] != -1)
    & (data["c_charge_degree"] != "O")
    & (data["score_text"] != "N/A")
]

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

#### Gender Distribution

In [ ]:
gender_counts = data['sex'].value_counts()
print("Gender distribution:\n", gender_counts)
sns.countplot(x='sex', data=data)
plt.title('Gender Distribution')
plt.show()

#### Race Distribution

In [ ]:
race_counts = data['race'].value_counts()
print("Race distribution:\n", gender_counts)
sns.countplot(x='race', data=data)
plt.title('Race Distribution')
plt.show()

In [18]:
# select two largest groups
data = data[(data["race"] == "African-American") | (data["race"] == "Caucasian")]

In [19]:
data['length_of_stay'] = (pd.to_datetime(data['c_jail_out']) - pd.to_datetime(data['c_jail_in'])).dt.days

In [20]:
# select columns
data = data[
    [
        "sex",
        "age",
        "race",
        "priors_count",
        "length_of_stay",
        "juv_fel_count",
        "juv_misd_count",
        "juv_other_count",
        "two_year_recid",
    ]
]

In [21]:
# Encode Sex, Male = 0 / Female = 1
data['sex'] = data['sex'].apply(lambda x: 0 if x == 'Male' else 1)

In [22]:
# Encode Race, African-American = 0 / Caucasian = 1
data['race'] = data['race'].apply(lambda x: 0 if x == 'African-American' else 1)

# Split the Data into Training and Testing

In [23]:
# define X and y
X = data.drop("two_year_recid", axis=1)
y = data["two_year_recid"]

# split the data in train-test sets; use random_state for reproducibility of the results
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# inspect dataset
display(X_train.head())

# proportion of positives
print("proportion of positives (train): %.2f" % y_train.mean())

In [24]:
# fairlearn
from fairlearn.metrics import (
    false_positive_rate,
    false_negative_rate,
    true_positive_rate,
    MetricFrame,
    equalized_odds_difference,
    demographic_parity_difference,
)

def score(
    y_train,
    y_train_pred,
    y_test,
    y_test_pred,
    sensitive_features_train,
    sensitive_features_test,
    metrics={"accuracy": accuracy_score, "fpr": false_positive_rate, "fnr": false_negative_rate,},
):
    """
    Helper function to evaluate classifiers without too much repetition of code.
    """

    # training set
    mf_train = MetricFrame(
        metrics=metrics,
        y_true=y_train,
        y_pred=y_train_pred,
        sensitive_features=sensitive_features_train,
    )

    # test set
    mf_test = MetricFrame(
        metrics=metrics,
        y_true=y_test,
        y_pred=y_test_pred,
        sensitive_features=sensitive_features_test,
    )

    # display results
    display(
        pd.concat(
            [mf_train.by_group, mf_test.by_group], keys=["train", "test"]
        ).unstack(level=0)
    )

    # compute metrics
    print(
        "equalized odds (test): %.2f"
        % equalized_odds_difference(
            y_true=y_test,
            y_pred=y_test_pred,
            sensitive_features=sensitive_features_test,
        )
    )

    print("accuracy (test): %.2f" % accuracy_score(y_true=y_test, y_pred=y_test_pred))
    return

# Random Forest Calssifier

In [28]:
naiveModel = RandomForestClassifier()
naiveModel.fit(X_train,y_train)
y_pred = naiveModel.predict(X_test)

In [11]:
filename = 'random_forest_classifier.sav'

In [ ]:
naiveModel = pickle.load(open('Compas/Models/random_forest_classifier.sav', 'rb'))
y_pred = naiveModel.predict(X_test)

# Prediction Report

In [ ]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
confusion = confusion_matrix(y_test, y_pred)
print(confusion)
sns.heatmap(confusion, annot=True, fmt=".2f")

# Test Which Features are Influencing the Prediction

In [ ]:
shap_explainer = shap.Explainer(naiveModel)
shap_values = shap_explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train, feature_names=data.columns)

# Test Model Fairness

In [ ]:
# score
score(
    y_train,
    naiveModel.predict(X_train),
    y_test,
    naiveModel.predict(X_test),
    X_train["race"],
    X_test["race"],
)

# Save the Model

In [ ]:
#Save the trained model
pickle.dump(naiveModel, open('Compas/Models/'+filename, 'wb'))

# Bias Mitigation

#### Load Fair Model

In [42]:
import dill

file_name = 'fair_model.pkl'

with open('Compas/Models/'+file_name, 'rb') as f:
    fair_model2 = dill.load(f)

y_pred_fair2 = fair_model2.predict(X_test)

#### In Process Using Exponentiated Gradient

In [ ]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds
# train model
fair_model = ExponentiatedGradient(
    estimator= RandomForestClassifier(),
    constraints= EqualizedOdds(),
    eps=0.01,
)
fair_model.fit(X=X_train, y=y_train, sensitive_features=X_train["race"])
y_pred_fair = fair_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_fair))
confusion = confusion_matrix(y_test, y_pred_fair)
print(confusion)
sns.heatmap(confusion, annot=True, fmt=".2f")
print(accuracy_score(y_test,y_pred_fair))

# Test Fairness of New Fair Model

In [ ]:
# score
score(
    y_train,
    fair_model.predict(X_train, random_state=0),
    y_test,
    fair_model.predict(X_test, random_state=0),
    X_train["race"],
    X_test["race"],
)

# Save Fair Model

In [ ]:
print(fair_model.sample_weight_name)

In [153]:
import dill

# Save the fair model
filename = 'fair_model.pkl'

dill.dump(fair_model, open('Compas/Models/'+file_name,'wb'))

# Prepare the Affected Dataset

In [ ]:
# Find instances where the model predicted target class 0
predicted_target_1_instances = X_test[y_pred_fair == 1]
print(predicted_target_1_instances['race'].value_counts())
#africanAmerican = predicted_target_1_instances[predicted_target_1_instances['race'] == 0].sample(n=1, random_state=42)  # 42 is a random seed
#caucasian = predicted_target_1_instances[predicted_target_1_instances['race'] == 1].sample(n=1, random_state=42)
#filtered_test_data = pd.concat([africanAmerican, caucasian])
#predicted_target_1_instances = filtered_test_data.sample(frac=1, random_state=42)

# Save the filtered instances to a new dataset
predicted_target_1_instances.to_csv('Compas/Dataset/predicted_target_1_instances.csv', index=False)

# RL Agent Training

In [ ]:
dataset = data
affected_dataset = predicted_target_1_instances
model = fair_model  #naiveModel
target = 0
protected_attribute = "race"
features_to_change = ["priors_count", "length_of_stay"]
number_of_counterfactuals = 5
minimums = [0, 0] #[0, 1, 0]
maximums = [38, 799]#[38, 10, 799]
explainer = Explainer(dataset, affected_dataset, model, protected_attribute, features_to_change, number_of_counterfactuals, target, minimums, maximums, action_effectiveness=0.7)
explainer.train()

x, y = explainer.plot()
plt.show()
plt.plot(x, y)
plt.xlabel("Number of Timesteps")
plt.ylabel("Rewards")
plt.title("Learning Curve" + " Smoothed")
plt.show()

cfs = explainer.report_counterfactuals()
print(cfs)

In [20]:
cfs_sorted = cfs.sort_values(by='Reward', ascending=False)
cfs_sorted.to_csv('Compas/Fair CF/fair_cf_new.csv', index=False)